In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings


##have pandas display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
 # Show all columns in df.info()

warnings.filterwarnings('ignore')

%matplotlib inline



In [2]:
df = pd.read_csv("../data/drug_data.csv")

In [3]:
df.head()

,prescription_date,prescription_year,prescription_month,day_of_week,quarter,patient_id,patient_taj,patient_name,birth_date,age_at_prescription,patient_gender,birth_place,medication_name,active_ingredient,atc_code,atc_level1,atc_level2,atc_level3,pharmaceutical_form,active_amount,active_unit,package_quantity,package_unit,ttt_code,ksz_code,package_size,dosage,prescription_status,has_status_data,is_filled,encounter_id,encounter_date,clinic_name,mep_code,medical_specialty,visit_reason,is_admin_visit,doctor_stamp,doctor_name,icd10_codes,diagnosis_names,num_diagnoses,has_diabetes_type1,has_diabetes_type2,has_diabetes_any,has_hyperlipidemia,has_obesity,has_hypothyroidism,has_hyperthyroidism,has_hypertension,has_angina,has_myocardial_infarction,has_ischemic_heart_disease,has_atrial_fibrillation,has_heart_failure,has_cerebrovascular_disease,has_stroke,has_peripheral_vascular_disease,has_asthma,has_copd,has_pneumonia,has_dementia,has_schizophrenia,has_mood_disorder,has_depression,has_anxiety_disorder,has_substance_abuse,has_rheumatoid_arthritis,has_osteoarthritis,has_osteoporosis,has_chronic_kidney_disease,has_acute_kidney_failure,has_cancer,has_breast_cancer,has_lung_cancer,has_colorectal_cancer,has_peptic_ulcer,has_inflammatory_bowel_disease,has_liver_disease,is_pregnant,has_injury,has_infectious_disease,has_blood_disorder,has_neurological_disorder,has_eye_disorder,has_ear_disorder,has_skin_disorder,has_symptoms_signs_abnormal_findings,data_source,prescription_number
0,2023-02-22 17:47:44.000,2023,2,3,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,ALGOPYRIN 500 MG TABLETTA,metamizol-nátrium,N02BB02,N,N02,N02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10x buborékcsomagolásban,1x1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82798,GYŐRI-DANI VERONIKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
1,2023-02-22 17:36:49.000,2023,2,3,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,LORDESTIN 5 MG FILMTABLETTA,desloratadin,R06AX27,R,R06,R06A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x buborékcsomagolásban,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
2,2023-02-22 17:36:49.000,2023,2,3,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,VITAMIN D3 PHARMA PATENT 30000 NE FILMTABLETTA,Colecalciferol,A11CC05,A,A11,A11C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2x PVC/PVdC // Alu buborékcsomagolásban, dobozban",utasítás szerint,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
3,2023-07-06 18:04:29.000,2023,7,4,3,H32001000004,105 568 760,Zámbó József,1966-08-01,56.0,férfi,Nyíregyháza,BÉRES CALCIUM 500 MG FILMTABLETTA,kalcium-karbonát,A12AA04,A,A12,A12A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x hdpe tartályban,D.S.: reggel: 1 tabletta naponta,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45875,SZABÓ LAJOS MIKLÓS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1
4,2023-12-18 13:11:07.000,2023,12,1,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,BISOPROLOL-RATIOPHARM 5 MG TABLETTA,bisoprolol,C07AB07,C,C07,C07A,tabletta,5.0,mg,30.0,db,210156821.0,30x buborékcsomagolásban,NaN,NaN,NaN,0,NaN,H32001007373,2023-12-18 12:57:39.000,Nyírkáta rendelő,NaN,Belgyógyászat,Akut ellátás,0.0,70282,Dr. Varga Beáta,"I10H0, R0000, Z0130","Magasvérnyomás-betegség (elsődleges), Tachycar...",3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45282 entries, 0 to 45281
Data columns (total 90 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   prescription_date                     45282 non-null  object 
 1   prescription_year                     45282 non-null  int64  
 2   prescription_month                    45282 non-null  int64  
 3   day_of_week                           45282 non-null  int64  
 4   quarter                               45282 non-null  int64  
 5   patient_id                            45282 non-null  object 
 6   patient_taj                           45270 non-null  object 
 7   patient_name                          45270 non-null  object 
 8   birth_date                            45270 non-null  object 
 9   age_at_prescription                   45270 non-null  float64
 10  patient_gender                        45270 non-null  object 
 11  birth_place    

## HANDLING DATETIME COLUMNS

In [5]:

df['prescription_date'] = pd.to_datetime(df['prescription_date'])


In [6]:
## drop time component from prescription_date
df['prescription_date_dateonly'] = pd.to_datetime(df['prescription_date'].dt.date)





In [7]:
## month_start = first day of that month (e.g., 2025-02-01)
df['month_start'] = df['prescription_date_dateonly'].dt.to_period('M').dt.start_time

In [8]:
## recalculate prescription_year from prescription_date_dateonly
df['prescription_year'] = df['prescription_date_dateonly'].dt.year
df['prescription_month'] = df['prescription_date_dateonly'].dt.month
df['day_of_week'] = df['prescription_date_dateonly'].dt.dayofweek
df['quarter'] = df['prescription_date_dateonly'].dt.quarter






## Normalize and map prescription_status

In [9]:
## normalize prescription_status, lowercase and trim and all strings

df['prescription_status_norm'] = df['prescription_status'].astype('str').str.lower().str.strip()

In [10]:
map_prescription_status = {
    "3": "filled",
    "4": "filled",
    "5":"filled",
    "8":"filled",
    "dispensed":"filled",
    "6":"not_filled",
    "withdrawn":"not_filled",
     "7":"not_filled",
    "open":"unknown",
    "1":"unknown",
    "2":"unknown",
}

df['prescription_filled_status'] = df['prescription_status_norm'].map(map_prescription_status)



In [11]:
df['prescription_filled_status'].value_counts()

prescription_filled_status
filled        23976
unknown        7579
not_filled     2575
Name: count, dtype: int64

In [12]:
df['prescription_filled_status'].isnull().sum()

np.int64(11152)

In [13]:
##kAdd known_status_flag = 1 if filled/not_filled else 0.
df['known_status_flag'] = df['prescription_filled_status'].isin(['filled','not_filled']).astype(int)


In [14]:
df['known_status_flag'].value_counts()

known_status_flag
1    26551
0    18731
Name: count, dtype: int64

In [15]:
## transform atc_level3 to string and uppercase
df['atc3_code'] = df['atc_level3'].astype('str').str.upper().str.strip()


In [16]:
len(df[df['atc3_code']=='NAN'])

3870

In [17]:
df['medication_name'] = df['medication_name'].str.strip().str.lower()







In [18]:
# 1️⃣ Count how many rows per (date, medication)
counts = (
    df.groupby(['patient_id', 'prescription_date_dateonly', 'medication_name'])
    .size()
    .reset_index(name='count')
)

# 2️⃣ Filter combinations that occur more than once
duplicates = counts[counts['count'] > 1]



In [19]:
df['drug_code_name'] = df['atc3_code'].replace(['', 'NAN', 'NaN', 'nan'], pd.NA)

# Group by patient and compute flags
consistency = (
    df.groupby('patient_id')
      .agg(
          has_atc3=('drug_code_name', lambda x: x.notna().any()),
          missing_atc3=('drug_code_name', lambda x: x.isna().any())
      )
      .reset_index()
)

# Patients with both present AND missing ATC3
mixed = consistency[consistency['has_atc3'] & consistency['missing_atc3']]

mixed.head()

,patient_id,has_atc3,missing_atc3
8,H32001000021,True,True
13,H32001000037,True,True
17,H32001000048,True,True
25,H32001000064,True,True
26,H32001000065,True,True


In [20]:
## create rx_line_id
df['drug_code_name'] = df['drug_code_name'].fillna(df['medication_name'])

df['rx_line_id'] = df['patient_id'].astype(str)+'_'+df['drug_code_name']+'_'+df['prescription_date_dateonly'].astype(str)

In [21]:
df = df.sort_values(['rx_line_id','prescription_date'])

In [22]:
df.head()

,prescription_date,prescription_year,prescription_month,day_of_week,quarter,patient_id,patient_taj,patient_name,birth_date,age_at_prescription,patient_gender,birth_place,medication_name,active_ingredient,atc_code,atc_level1,atc_level2,atc_level3,pharmaceutical_form,active_amount,active_unit,package_quantity,package_unit,ttt_code,ksz_code,package_size,dosage,prescription_status,has_status_data,is_filled,encounter_id,encounter_date,clinic_name,mep_code,medical_specialty,visit_reason,is_admin_visit,doctor_stamp,doctor_name,icd10_codes,diagnosis_names,num_diagnoses,has_diabetes_type1,has_diabetes_type2,has_diabetes_any,has_hyperlipidemia,has_obesity,has_hypothyroidism,has_hyperthyroidism,has_hypertension,has_angina,has_myocardial_infarction,has_ischemic_heart_disease,has_atrial_fibrillation,has_heart_failure,has_cerebrovascular_disease,has_stroke,has_peripheral_vascular_disease,has_asthma,has_copd,has_pneumonia,has_dementia,has_schizophrenia,has_mood_disorder,has_depression,has_anxiety_disorder,has_substance_abuse,has_rheumatoid_arthritis,has_osteoarthritis,has_osteoporosis,has_chronic_kidney_disease,has_acute_kidney_failure,has_cancer,has_breast_cancer,has_lung_cancer,has_colorectal_cancer,has_peptic_ulcer,has_inflammatory_bowel_disease,has_liver_disease,is_pregnant,has_injury,has_infectious_disease,has_blood_disorder,has_neurological_disorder,has_eye_disorder,has_ear_disorder,has_skin_disorder,has_symptoms_signs_abnormal_findings,data_source,prescription_number,prescription_date_dateonly,month_start,prescription_status_norm,prescription_filled_status,known_status_flag,atc3_code,drug_code_name,rx_line_id
2,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,vitamin d3 pharma patent 30000 ne filmtabletta,Colecalciferol,A11CC05,A,A11,A11C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2x PVC/PVdC // Alu buborékcsomagolásban, dobozban",utasítás szerint,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,A11C,A11C,H32001000001_A11C_2023-02-22
0,2023-02-22 17:47:44,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,algopyrin 500 mg tabletta,metamizol-nátrium,N02BB02,N,N02,N02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10x buborékcsomagolásban,1x1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82798,GYŐRI-DANI VERONIKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,6,not_filled,1,N02B,N02B,H32001000001_N02B_2023-02-22
1,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,lordestin 5 mg filmtabletta,desloratadin,R06AX27,R,R06,R06A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x buborékcsomagolásban,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,R06A,R06A,H32001000001_R06A_2023-02-22
8,2023-12-18 13:09:22,2023,12,0,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,esomeprazol sandoz 40 mg gyomornedv-ellenálló ...,esomeprazol,A02BC05,A,A02,A02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28x buborékcsomagolásban alu//alu,D.S.: reggel: 1 tabletta naponta,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45875,SZABÓ LAJOS MIKLÓS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,2,2023-12-18,2023-12-01,1,unknown,0,A02B,A02B,H32001000004_A02B_2023-12-18
9,2023-12-18 13:09:22,2023,12,0,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,es

In [23]:
df['status_norm'] = (
    df['prescription_filled_status']
      .astype(str)
      .str.strip()
      .str.lower()
)
FINAL_STATUSES = ['filled', 'not_filled']

In [24]:
def get_day_summary(group):
    n_filled     = (group['status_norm'] == 'filled').sum()
    n_not_filled = (group['status_norm'] == 'not_filled').sum()

    if n_not_filled > 0:
        final_status = 'not_filled'
    elif n_filled > 0:
        final_status = 'filled'
    else:
        final_status = 'unknown'
    
    # pick latest representative row
    if final_status == 'unknown':
        rep = group.iloc[group['prescription_date'].argmax()]
    else:
        rep = group[group['status_norm'] == final_status] \
                  .iloc[group[group['status_norm'] == final_status]['prescription_date'].argmax()]

    return pd.Series({
        'final_status_day': final_status,
        'day_entries_merged_count': len(group),
        'packages_that_day': n_filled,
        'had_conflict_final_same_day': int((n_filled > 0) and (n_not_filled > 0)),
        'prescription_date_rep': rep['prescription_date'],
        'known_status_flag_rep': rep['known_status_flag']
    })

# Attach day-level fields directly to df
df_day_summary = (
    df.groupby(['rx_line_id', 'prescription_date_dateonly'])
      .apply(get_day_summary)
      .reset_index()
)


In [25]:
df = df.merge(
    df_day_summary,
    on=['rx_line_id', 'prescription_date_dateonly'],
    how='left'
)

df.head()

,prescription_date,prescription_year,prescription_month,day_of_week,quarter,patient_id,patient_taj,patient_name,birth_date,age_at_prescription,patient_gender,birth_place,medication_name,active_ingredient,atc_code,atc_level1,atc_level2,atc_level3,pharmaceutical_form,active_amount,active_unit,package_quantity,package_unit,ttt_code,ksz_code,package_size,dosage,prescription_status,has_status_data,is_filled,encounter_id,encounter_date,clinic_name,mep_code,medical_specialty,visit_reason,is_admin_visit,doctor_stamp,doctor_name,icd10_codes,diagnosis_names,num_diagnoses,has_diabetes_type1,has_diabetes_type2,has_diabetes_any,has_hyperlipidemia,has_obesity,has_hypothyroidism,has_hyperthyroidism,has_hypertension,has_angina,has_myocardial_infarction,has_ischemic_heart_disease,has_atrial_fibrillation,has_heart_failure,has_cerebrovascular_disease,has_stroke,has_peripheral_vascular_disease,has_asthma,has_copd,has_pneumonia,has_dementia,has_schizophrenia,has_mood_disorder,has_depression,has_anxiety_disorder,has_substance_abuse,has_rheumatoid_arthritis,has_osteoarthritis,has_osteoporosis,has_chronic_kidney_disease,has_acute_kidney_failure,has_cancer,has_breast_cancer,has_lung_cancer,has_colorectal_cancer,has_peptic_ulcer,has_inflammatory_bowel_disease,has_liver_disease,is_pregnant,has_injury,has_infectious_disease,has_blood_disorder,has_neurological_disorder,has_eye_disorder,has_ear_disorder,has_skin_disorder,has_symptoms_signs_abnormal_findings,data_source,prescription_number,prescription_date_dateonly,month_start,prescription_status_norm,prescription_filled_status,known_status_flag,atc3_code,drug_code_name,rx_line_id,status_norm,final_status_day,day_entries_merged_count,packages_that_day,had_conflict_final_same_day,prescription_date_rep,known_status_flag_rep
0,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,vitamin d3 pharma patent 30000 ne filmtabletta,Colecalciferol,A11CC05,A,A11,A11C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2x PVC/PVdC // Alu buborékcsomagolásban, dobozban",utasítás szerint,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,A11C,A11C,H32001000001_A11C_2023-02-22,filled,filled,1,1,0,2023-02-22 17:36:49,1
1,2023-02-22 17:47:44,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,algopyrin 500 mg tabletta,metamizol-nátrium,N02BB02,N,N02,N02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10x buborékcsomagolásban,1x1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82798,GYŐRI-DANI VERONIKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,6,not_filled,1,N02B,N02B,H32001000001_N02B_2023-02-22,not_filled,not_filled,1,0,0,2023-02-22 17:47:44,1
2,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,lordestin 5 mg filmtabletta,desloratadin,R06AX27,R,R06,R06A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x buborékcsomagolásban,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,R06A,R06A,H32001000001_R06A_2023-02-22,filled,filled,1,1,0,2023-02-22 17:36:49,1
3,2023-12-18 13:09:22,2023,12,0,4,H32001000004,105 568 760,Zámbó József,1966-08-01,57.0,férfi,Nyíregyháza,esomeprazol sandoz 40 mg gyomornedv-ellenálló ...,esomeprazol,A02BC05,A,A02,A02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28x buborékcsomagolásban alu//alu,D.S.: reggel: 1 tabletta naponta,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45875,SZABÓ LAJOS MIKLÓS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [26]:
def get_line_summary(group):
    finals = group[group['final_status_day'].isin(['filled', 'not_filled'])]

    if len(finals) == 0:
        chosen = group.loc[group['prescription_date_rep'].idxmax()]
    else:
        chosen = finals.loc[finals['prescription_date_rep'].idxmax()]
    
    return pd.Series({
        'final_status': chosen['final_status_day'],
        'prescription_date_final': chosen['prescription_date_dateonly'],
        'prescription_datetime_final': chosen['prescription_date_rep'],
        'entries_merged_total': group['day_entries_merged_count'].sum(),
        'had_conflict_any_day': int(group['had_conflict_final_same_day'].any()),
        'packages_total': group['packages_that_day'].sum(),
        'known_status_flag_final': chosen['known_status_flag_rep'],
    })

df_line_summary = (
    df.groupby('rx_line_id')
      .apply(get_line_summary)
      .reset_index()
)


In [27]:
df = df.merge(df_line_summary, on='rx_line_id', how='left')


In [28]:
df = df.drop_duplicates(subset=['rx_line_id'])

In [29]:
df.shape

(27206, 112)

In [30]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 27206 entries, 0 to 45281
Data columns (total 112 columns):
 #    Column                                Dtype         
---   ------                                -----         
 0    prescription_date                     datetime64[ns]
 1    prescription_year                     int32         
 2    prescription_month                    int32         
 3    day_of_week                           int32         
 4    quarter                               int32         
 5    patient_id                            object        
 6    patient_taj                           object        
 7    patient_name                          object        
 8    birth_date                            object        
 9    age_at_prescription                   float64       
 10   patient_gender                        object        
 11   birth_place                           object        
 12   medication_name                       object        
 13   acti

In [31]:
df['final_status'] = (
    df['final_status']
      .astype(str)
      .str.strip()
      .str.lower()
)

In [32]:
known_vals = ['filled', 'not_filled']

monthly = (
    df
    .groupby('month_start')
    .agg(
        total_rx=('rx_line_id', 'size'),
        # how many lines have known final status
        known_status_rx=('known_status_flag_final', 'sum'),
        # how many are filled
        filled_rx=('final_status', lambda x: (x == 'filled').sum())
    )
    .reset_index()
)

In [33]:
monthly['status_coverage'] = monthly['known_status_rx'] / monthly['total_rx']
monthly.loc[monthly['total_rx'] == 0, 'status_coverage'] = pd.NA

# refill_adherence = filled_rx / known_status_rx  (NA if known_status_rx == 0)
monthly['refill_adherence'] = monthly['filled_rx'] / monthly['known_status_rx']
monthly.loc[monthly['known_status_rx'] == 0, 'refill_adherence'] = pd.NA

In [34]:
monthly.head(10)

,month_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence
0,2023-02-01,6,3,2,0.500000,0.666667
1,2023-03-01,294,266,225,0.904762,0.845865
2,2023-04-01,499,450,412,0.901804,0.915556
3,2023-05-01,992,723,664,0.728831,0.918396
4,2023-06-01,797,541,448,0.678795,0.828096
5,2023-07-01,764,555,503,0.726440,0.906306
6,2023-08-01,951,670,592,0.704522,0.883582
7,2023-09-01,1473,1043,908,0.708079,0.870566
8,2023-10-01,1570,1070,955,0.681529,0.892523
9,2023-11-01,1025,570,500,0.556098,0.877193


In [35]:
monthly['unique_patients'] = monthly['month_start'].map(
    df.groupby('month_start')['patient_id'].nunique()
)   

In [36]:
monthly.head(10)

,month_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients
0,2023-02-01,6,3,2,0.500000,0.666667,3
1,2023-03-01,294,266,225,0.904762,0.845865,52
2,2023-04-01,499,450,412,0.901804,0.915556,78
3,2023-05-01,992,723,664,0.728831,0.918396,172
4,2023-06-01,797,541,448,0.678795,0.828096,173
5,2023-07-01,764,555,503,0.726440,0.906306,151
6,2023-08-01,951,670,592,0.704522,0.883582,182
7,2023-09-01,1473,1043,908,0.708079,0.870566,207
8,2023-10-01,1570,1070,955,0.681529,0.892523,227
9,2023-11-01,1025,570,500,0.556098,0.877193,251


In [37]:
df['patient_gender'].value_counts()


patient_gender
nő       19386
férfi     7808
Name: count, dtype: int64

In [38]:
## map gender to 'female' 'male' and 'unknown'
gender_map = {
    'nő': 'female',
    'férfi': 'male',
    'nan': 'unknown',
}

df['gender_norm'] = df['patient_gender'].str.strip().str.lower().map(gender_map)


In [39]:
df['gender_norm'].value_counts(dropna=False)

gender_norm
female    19386
male       7808
NaN          12
Name: count, dtype: int64

In [40]:
gender_stats = (
    df
    .groupby('month_start')
    .agg(
        female_count=('gender_norm', lambda x: (x == 'female').sum()),
        known_gender_count=('gender_norm', lambda x: x.notna().sum())
    )
)

gender_stats['pct_female'] = (
    gender_stats['female_count'] / gender_stats['known_gender_count']
)

# If no known gender that month → NA
gender_stats.loc[gender_stats['known_gender_count'] == 0, 'pct_female'] = pd.NA

In [41]:
monthly['pct_female'] = monthly['month_start'].map(gender_stats['pct_female'])


In [42]:
monthly.head(10)

,month_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female
0,2023-02-01,6,3,2,0.500000,0.666667,3,1.000000
1,2023-03-01,294,266,225,0.904762,0.845865,52,0.656463
2,2023-04-01,499,450,412,0.901804,0.915556,78,0.779559
3,2023-05-01,992,723,664,0.728831,0.918396,172,0.684476
4,2023-06-01,797,541,448,0.678795,0.828096,173,0.744040
5,2023-07-01,764,555,503,0.726440,0.906306,151,0.759162
6,2023-08-01,951,670,592,0.704522,0.883582,182,0.813880
7,2023-09-01,1473,1043,908,0.708079,0.870566,207,0.742023
8,2023-10-01,1570,1070,955,0.681529,0.892523,227,0.674522
9,2023-11-01,1025,570,500,0.556098,0.877193,251,0.762927


In [43]:
age_stats = (
    df.groupby('month_start')
    .agg(
        median_age=('age_at_prescription', lambda x: x.dropna().median())
    )
)

In [44]:
monthly['median_age'] = monthly['month_start'].map(age_stats['median_age'])

In [45]:
monthly.tail(10)

,month_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female,median_age
24,2025-02-01,440,0,0,0.0,NaN,247,0.656818,55.0
25,2025-03-01,405,0,0,0.0,NaN,224,0.716049,56.0
26,2025-04-01,419,0,0,0.0,NaN,223,0.687351,57.0
27,2025-05-01,427,0,0,0.0,NaN,231,0.688525,58.0
28,2025-06-01,437,0,0,0.0,NaN,215,0.700229,58.0
29,2025-07-01,327,0,0,0.0,NaN,184,0.639144,55.0
30,2025-08-01,343,0,0,0.0,NaN,192,0.676385,58.0
31,2025-09-01,372,0,0,0.0,NaN,210,0.712366,57.0
32,2025-10-01,468,0,0,0.0,NaN,248,0.735043,55.0
33,2025-11-01,135,0,0,0.0,NaN,66,0.674074,56.0


In [46]:
monthly['low_coverage_flag'] = (monthly['status_coverage'] < 0.70).astype(int)

In [47]:
monthly.head(10)

,month_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female,median_age,low_coverage_flag
0,2023-02-01,6,3,2,0.500000,0.666667,3,1.000000,26.0,1
1,2023-03-01,294,266,225,0.904762,0.845865,52,0.656463,57.0,0
2,2023-04-01,499,450,412,0.901804,0.915556,78,0.779559,59.0,0
3,2023-05-01,992,723,664,0.728831,0.918396,172,0.684476,58.0,0
4,2023-06-01,797,541,448,0.678795,0.828096,173,0.744040,60.0,1
5,2023-07-01,764,555,503,0.726440,0.906306,151,0.759162,58.5,0
6,2023-08-01,951,670,592,0.704522,0.883582,182,0.813880,60.0,0
7,2023-09-01,1473,1043,908,0.708079,0.870566,207,0.742023,61.0,0
8,2023-10-01,1570,1070,955,0.681529,0.892523,227,0.674522,59.0,1
9,2023-11-01,1025,570,500,0.556098,0.877193,251,0.762927,57.0,1


In [48]:
known_per_month = (
    df.groupby('month_start')['known_status_flag_final']
      .sum()
      .reset_index(name='known_status_rx')
)

In [49]:
monthly_overall = monthly.copy()

In [50]:
monthly_overall['month_start'] = pd.to_datetime(monthly_overall['month_start'])

In [51]:
monthly_calendar = pd.date_range(start=monthly_overall['month_start'].min(), end=monthly_overall['month_start'].max(),freq='MS')

In [52]:
monthly_calendar

DatetimeIndex(['2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01',
               '2023-06-01', '2023-07-01', '2023-08-01', '2023-09-01',
               '2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01',
               '2024-02-01', '2024-03-01', '2024-04-01', '2024-05-01',
               '2024-06-01', '2024-07-01', '2024-08-01', '2024-09-01',
               '2024-10-01', '2024-11-01', '2024-12-01', '2025-01-01',
               '2025-02-01', '2025-03-01', '2025-04-01', '2025-05-01',
               '2025-06-01', '2025-07-01', '2025-08-01', '2025-09-01',
               '2025-10-01', '2025-11-01'],
              dtype='datetime64[ns]', freq='MS')

In [53]:
monthly_full = (
    monthly_overall
    .set_index('month_start')
    .reindex(monthly_calendar)   # reindex to full monthly calendar
)

In [54]:
monthly_full.index.name = 'month_start'

In [55]:
count_cols = ['total_rx', 'known_status_rx', 'filled_rx', 'unique_patients']

for col in count_cols:
    if col in monthly_full.columns:
        monthly_full[col] = monthly_full[col].fillna(0).astype(int)

In [56]:
if {'status_coverage', 'refill_adherence'}.issubset(monthly_full.columns):
    # status_coverage = known_status_rx / total_rx
    monthly_full['status_coverage'] = (
        monthly_full['known_status_rx'] / monthly_full['total_rx']
    )
    monthly_full.loc[monthly_full['total_rx'] == 0, 'status_coverage'] = pd.NA

    # refill_adherence = filled_rx / known_status_rx
    monthly_full['refill_adherence'] = (
        monthly_full['filled_rx'] / monthly_full['known_status_rx']
    )
    monthly_full.loc[monthly_full['known_status_rx'] == 0, 'refill_adherence'] = pd.NA

In [57]:
monthly_full = monthly_full.reset_index()

In [58]:
monthly_full.shape

(34, 10)

In [59]:
## drop rows from 2025-02-01 and onwards
monthly_full = monthly_full[monthly_full['month_start'] < '2025-02-01']
monthly_full.head(10)



,month_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female,median_age,low_coverage_flag
0,2023-02-01,6,3,2,0.500000,0.666667,3,1.000000,26.0,1
1,2023-03-01,294,266,225,0.904762,0.845865,52,0.656463,57.0,0
2,2023-04-01,499,450,412,0.901804,0.915556,78,0.779559,59.0,0
3,2023-05-01,992,723,664,0.728831,0.918396,172,0.684476,58.0,0
4,2023-06-01,797,541,448,0.678795,0.828096,173,0.744040,60.0,1
5,2023-07-01,764,555,503,0.726440,0.906306,151,0.759162,58.5,0
6,2023-08-01,951,670,592,0.704522,0.883582,182,0.813880,60.0,0
7,2023-09-01,1473,1043,908,0.708079,0.870566,207,0.742023,61.0,0
8,2023-10-01,1570,1070,955,0.681529,0.892523,227,0.674522,59.0,1
9,2023-11-01,1025,570,500,0.556098,0.877193,251,0.762927,57.0,1


In [60]:
monthly_full.shape

(24, 10)

In [61]:
## save in data as monthly_overall.csv
monthly_full.to_csv('../data/monthly_overall.csv', index=False)


In [62]:
df['atc3_code'].value_counts(dropna=False)

atc3_code
NAN     2498
C10A    2016
A02B    1663
C09B    1631
M01A    1447
C07A    1388
A10B     969
B01A     950
C09A     901
N05B     803
A11C     769
C08C     569
R03A     426
R06A     403
N02A     381
J01C     379
B03A     376
N02B     339
J01F     332
N06B     329
M04A     305
M03B     303
C09C     282
C09D     272
R03D     271
H03A     271
C02A     268
C03C     262
C03B     257
ISMT     244
J01M     232
M02A     205
A12B     195
B03B     190
C01E     185
J01D     183
J01X     178
C03D     167
C01D     166
N06A     165
A12C     164
N07C     153
C02C     138
C10B     130
R01A     129
R03B     120
A03A     119
N05C     116
A12A     115
A10A     113
G04C     108
N03A     108
C04A     104
A11D     102
D07A     100
J01E     100
S01A      83
A03F      81
R05D      78
S01E      72
D01A      67
G03A      67
A16A      65
R05C      63
H03B      63
C05C      60
A11E      56
H02A      55
G04B      53
D07C      51
J05A      50
J07B      44
G01A      43
D06A      43
N05A      43
C03E      42
C0

In [63]:
df['atc_level3_for_grouping'] = (
    df['atc3_code']
      .replace(['', 'NAN', 'NaN', 'nan'], pd.NA)   # normalize fake missing
      .fillna('ATC_UNKNOWN')                      # fill with fallback
)

In [64]:
df['atc_level3_for_grouping'].value_counts(dropna=False)

atc_level3_for_grouping
ATC_UNKNOWN    2498
C10A           2016
A02B           1663
C09B           1631
M01A           1447
C07A           1388
A10B            969
B01A            950
C09A            901
N05B            803
A11C            769
C08C            569
R03A            426
R06A            403
N02A            381
J01C            379
B03A            376
N02B            339
J01F            332
N06B            329
M04A            305
M03B            303
C09C            282
C09D            272
R03D            271
H03A            271
C02A            268
C03C            262
C03B            257
ISMT            244
J01M            232
M02A            205
A12B            195
B03B            190
C01E            185
J01D            183
J01X            178
C03D            167
C01D            166
N06A            165
A12C            164
N07C            153
C02C            138
C10B            130
R01A            129
R03B            120
A03A            119
N05C            116
A12A            

In [65]:
df_copy = df.copy()

In [66]:
df_copy['patient_gender'].value_counts(dropna=False)

patient_gender
nő       19386
férfi     7808
NaN         12
Name: count, dtype: int64

In [67]:
df_copy['final_status'] = (
    df_copy['final_status']
      .astype(str)
      .str.strip()
      .str.lower()
)

In [68]:
gender_map = {
    'nő': 'female',
    'férfi': 'male',
}
df_copy['gender_norm'] = df_copy['patient_gender'].map(gender_map)

In [69]:
known_status_vals = ['filled', 'not_filled']

atc_grouped = (
    df_copy
    .groupby(['month_start', 'atc_level3_for_grouping'])
    .agg(
        # Counts
        total_rx_atc3=('rx_line_id', 'size'),
        known_status_rx_atc3=('known_status_flag_final', 'sum'),
        filled_rx_atc3=('final_status', lambda x: (x == 'filled').sum()),
        unique_patients_atc3=('patient_id', 'nunique'),

        # Female share
        female_count_atc3=('gender_norm', lambda x: (x == 'female').sum()),
        known_gender_count_atc3=('gender_norm', lambda x: x.notna().sum()),

        # Median age among known ages
        median_age_atc3=('age_at_prescription', lambda x: x.dropna().median()),
    )
    .reset_index()
)

In [70]:
# status_coverage_atc3 = known / total
atc_grouped['status_coverage_atc3'] = (
    atc_grouped['known_status_rx_atc3'] / atc_grouped['total_rx_atc3']
)
atc_grouped.loc[atc_grouped['total_rx_atc3'] == 0, 'status_coverage_atc3'] = pd.NA

# refill_adherence_atc3 = filled / known
atc_grouped['refill_adherence_atc3'] = (
    atc_grouped['filled_rx_atc3'] / atc_grouped['known_status_rx_atc3']
)
atc_grouped.loc[atc_grouped['known_status_rx_atc3'] == 0, 'refill_adherence_atc3'] = pd.NA

# pct_female_atc3 = female / known_gender
atc_grouped['pct_female_atc3'] = (
    atc_grouped['female_count_atc3'] / atc_grouped['known_gender_count_atc3']
)
atc_grouped.loc[atc_grouped['known_gender_count_atc3'] == 0, 'pct_female_atc3'] = pd.NA

# low_coverage_flag_atc3 = 1 if status_coverage_atc3 < 0.70 else 0 (NA stays NA)
atc_grouped['low_coverage_flag_atc3'] = (
    (atc_grouped['status_coverage_atc3'] < 0.70)
    .astype('Int64')   # allows <NA>
)

In [71]:
monthly_atc = df_copy.merge(
    atc_grouped,
    on=['month_start', 'atc_level3_for_grouping'],
    how='left'
)

In [72]:
monthly_atc.head(10)

,prescription_date,prescription_year,prescription_month,day_of_week,quarter,patient_id,patient_taj,patient_name,birth_date,age_at_prescription,patient_gender,birth_place,medication_name,active_ingredient,atc_code,atc_level1,atc_level2,atc_level3,pharmaceutical_form,active_amount,active_unit,package_quantity,package_unit,ttt_code,ksz_code,package_size,dosage,prescription_status,has_status_data,is_filled,encounter_id,encounter_date,clinic_name,mep_code,medical_specialty,visit_reason,is_admin_visit,doctor_stamp,doctor_name,icd10_codes,diagnosis_names,num_diagnoses,has_diabetes_type1,has_diabetes_type2,has_diabetes_any,has_hyperlipidemia,has_obesity,has_hypothyroidism,has_hyperthyroidism,has_hypertension,has_angina,has_myocardial_infarction,has_ischemic_heart_disease,has_atrial_fibrillation,has_heart_failure,has_cerebrovascular_disease,has_stroke,has_peripheral_vascular_disease,has_asthma,has_copd,has_pneumonia,has_dementia,has_schizophrenia,has_mood_disorder,has_depression,has_anxiety_disorder,has_substance_abuse,has_rheumatoid_arthritis,has_osteoarthritis,has_osteoporosis,has_chronic_kidney_disease,has_acute_kidney_failure,has_cancer,has_breast_cancer,has_lung_cancer,has_colorectal_cancer,has_peptic_ulcer,has_inflammatory_bowel_disease,has_liver_disease,is_pregnant,has_injury,has_infectious_disease,has_blood_disorder,has_neurological_disorder,has_eye_disorder,has_ear_disorder,has_skin_disorder,has_symptoms_signs_abnormal_findings,data_source,prescription_number,prescription_date_dateonly,month_start,prescription_status_norm,prescription_filled_status,known_status_flag,atc3_code,drug_code_name,rx_line_id,status_norm,final_status_day,day_entries_merged_count,packages_that_day,had_conflict_final_same_day,prescription_date_rep,known_status_flag_rep,final_status,prescription_date_final,prescription_datetime_final,entries_merged_total,had_conflict_any_day,packages_total,known_status_flag_final,gender_norm,atc_level3_for_grouping,total_rx_atc3,known_status_rx_atc3,filled_rx_atc3,unique_patients_atc3,female_count_atc3,known_gender_count_atc3,median_age_atc3,status_coverage_atc3,refill_adherence_atc3,pct_female_atc3,low_coverage_flag_atc3
0,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,vitamin d3 pharma patent 30000 ne filmtabletta,Colecalciferol,A11CC05,A,A11,A11C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"2x PVC/PVdC // Alu buborékcsomagolásban, dobozban",utasítás szerint,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,3,filled,1,A11C,A11C,H32001000001_A11C_2023-02-22,filled,filled,1,1,0,2023-02-22 17:36:49,1,filled,2023-02-22,2023-02-22 17:36:49,1,0,1,1,NaN,A11C,1,1,1,1,0,0,NaN,1.000000,1.000000,NaN,0
1,2023-02-22 17:47:44,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,algopyrin 500 mg tabletta,metamizol-nátrium,N02BB02,N,N02,N02B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10x buborékcsomagolásban,1x1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82798,GYŐRI-DANI VERONIKA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EESZT,1,2023-02-22,2023-02-01,6,not_filled,1,N02B,N02B,H32001000001_N02B_2023-02-22,not_filled,not_filled,1,0,0,2023-02-22 17:47:44,1,not_filled,2023-02-22,2023-02-22 17:47:44,1,0,0,1,NaN,N02B,1,1,0,1,0,0,NaN,1.000000,0.000000,NaN,0
2,2023-02-22 17:36:49,2023,2,2,1,H32001000001,NaN,NaN,NaN,NaN,NaN,NaN,lordestin 5 mg filmtabletta,desloratadin,R06AX27,R,R06,R06A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30x buborékcsomagolásban,NaN,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59711,LENGYEL ANNA ANDREA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [73]:
atc_grouped = atc_grouped.drop(
    columns=['female_count_atc3', 'known_gender_count_atc3']
)

monthly_atc = atc_grouped[
    [
        'month_start',
        'atc_level3_for_grouping',
        'total_rx_atc3',
        'known_status_rx_atc3',
        'filled_rx_atc3',
        'status_coverage_atc3',
        'refill_adherence_atc3',
        'unique_patients_atc3',
        'pct_female_atc3',
        'median_age_atc3',
        'low_coverage_flag_atc3',
    ]
]

In [74]:
monthly_atc.head()

,month_start,atc_level3_for_grouping,total_rx_atc3,known_status_rx_atc3,filled_rx_atc3,status_coverage_atc3,refill_adherence_atc3,unique_patients_atc3,pct_female_atc3,median_age_atc3,low_coverage_flag_atc3
0,2023-02-01,A11C,1,1,1,1.0,1.0,1,NaN,NaN,0
1,2023-02-01,J01C,1,0,0,0.0,NaN,1,1.0,26.0,1
2,2023-02-01,M01A,1,0,0,0.0,NaN,1,NaN,NaN,1
3,2023-02-01,N02B,1,1,0,1.0,0.0,1,NaN,NaN,0
4,2023-02-01,R01A,1,0,0,0.0,NaN,1,NaN,NaN,1


In [75]:
monthly_atc['month_start'] = pd.to_datetime(monthly_atc['month_start'])

# All months from first to last (month start)
month_calendar = pd.date_range(
    start=monthly_atc['month_start'].min(),
    end=monthly_atc['month_start'].max(),
    freq='MS'   # Month Start
)

# All ATC3 classes we care about
atc_list = monthly_atc['atc_level3_for_grouping'].unique()

# Full (month, ATC3) index
full_index = pd.MultiIndex.from_product(
    [month_calendar, atc_list],
    names=['month_start', 'atc_level3_for_grouping']
)

In [76]:
monthly_atc_full = (
    monthly_atc
    .set_index(['month_start', 'atc_level3_for_grouping'])
    .reindex(full_index)
    .reset_index()
)

In [77]:
count_cols = [
    'total_rx_atc3',
    'known_status_rx_atc3',
    'filled_rx_atc3',
    'unique_patients_atc3',
]

for col in count_cols:
    monthly_atc_full[col] = monthly_atc_full[col].fillna(0).astype('int64')

In [78]:
# status_coverage_atc3 = known / total, NA if total == 0
monthly_atc_full['status_coverage_atc3'] = (
    monthly_atc_full['known_status_rx_atc3'] /
    monthly_atc_full['total_rx_atc3']
)
monthly_atc_full.loc[
    monthly_atc_full['total_rx_atc3'] == 0,
    'status_coverage_atc3'
] = pd.NA

# refill_adherence_atc3 = filled / known, NA if known == 0
monthly_atc_full['refill_adherence_atc3'] = (
    monthly_atc_full['filled_rx_atc3'] /
    monthly_atc_full['known_status_rx_atc3']
)
monthly_atc_full.loc[
    monthly_atc_full['known_status_rx_atc3'] == 0,
    'refill_adherence_atc3'
] = pd.NA

In [79]:
mask_low = monthly_atc_full['status_coverage_atc3'] < 0.70

monthly_atc_full['low_coverage_flag_atc3'] = (
    mask_low.where(monthly_atc_full['status_coverage_atc3'].notna())
            .astype('Int64')   # allows <NA>
)

In [80]:
monthly_atc_full.head()

,month_start,atc_level3_for_grouping,total_rx_atc3,known_status_rx_atc3,filled_rx_atc3,status_coverage_atc3,refill_adherence_atc3,unique_patients_atc3,pct_female_atc3,median_age_atc3,low_coverage_flag_atc3
0,2023-02-01,A11C,1,1,1,1.0,1.0,1,NaN,NaN,0
1,2023-02-01,J01C,1,0,0,0.0,NaN,1,1.0,26.0,1
2,2023-02-01,M01A,1,0,0,0.0,NaN,1,NaN,NaN,1
3,2023-02-01,N02B,1,1,0,1.0,0.0,1,NaN,NaN,0
4,2023-02-01,R01A,1,0,0,0.0,NaN,1,NaN,NaN,1


In [81]:
atc_sums = (
    monthly_atc_full
    .groupby('month_start')
    .agg(
        total_rx_atc3_sum=('total_rx_atc3', 'sum'),
        known_status_rx_atc3_sum=('known_status_rx_atc3', 'sum'),
        filled_rx_atc3_sum=('filled_rx_atc3', 'sum'),
    )
    .reset_index()
)


In [82]:
check = (
    monthly_overall
    .merge(atc_sums, on='month_start', how='left')
)


In [83]:
check['total_rx_match'] = (
    check['total_rx'] == check['total_rx_atc3_sum']
)

check['known_status_rx_match'] = (
    check['known_status_rx'] == check['known_status_rx_atc3_sum']
)

check['filled_rx_match'] = (
    check['filled_rx'] == check['filled_rx_atc3_sum']
)


In [84]:
count_mismatches = check[
    ~(check['total_rx_match'] &
      check['known_status_rx_match'] &
      check['filled_rx_match'])
]


In [85]:
count_mismatches

,month_start,total_rx,known_status_rx,filled_rx,status_coverage,refill_adherence,unique_patients,pct_female,median_age,low_coverage_flag,total_rx_atc3_sum,known_status_rx_atc3_sum,filled_rx_atc3_sum,total_rx_match,known_status_rx_match,filled_rx_match


In [86]:
monthly_atc.head(10)

,month_start,atc_level3_for_grouping,total_rx_atc3,known_status_rx_atc3,filled_rx_atc3,status_coverage_atc3,refill_adherence_atc3,unique_patients_atc3,pct_female_atc3,median_age_atc3,low_coverage_flag_atc3
0,2023-02-01,A11C,1,1,1,1.000000,1.000000,1,NaN,NaN,0
1,2023-02-01,J01C,1,0,0,0.000000,NaN,1,1.000000,26.0,1
2,2023-02-01,M01A,1,0,0,0.000000,NaN,1,NaN,NaN,1
3,2023-02-01,N02B,1,1,0,1.000000,0.000000,1,NaN,NaN,0
4,2023-02-01,R01A,1,0,0,0.000000,NaN,1,NaN,NaN,1
5,2023-02-01,R06A,1,1,1,1.000000,1.000000,1,NaN,NaN,0
6,2023-03-01,A02B,22,21,19,0.954545,0.904762,20,0.681818,51.0,0
7,2023-03-01,A03A,3,3,3,1.000000,1.000000,3,0.333333,63.0,0
8,2023-03-01,A03F,1,1,1,1.000000,1.000000,1,0.000000,47.0,0
9,2023-03-01,A07E,1,1,1,1.000000,1.000000,1,0.000000,83.0,0


In [87]:
g = monthly_atc.groupby('atc_level3_for_grouping')

atc_summary = g.agg(
    n_months=('month_start', 'nunique'),
    median_known_status_rx_atc3=('known_status_rx_atc3', 'median'),
    median_status_coverage_atc3=('status_coverage_atc3', 'median'),
    months_zero_total=('total_rx_atc3', lambda x: (x == 0).sum()),
)

# missing fraction
atc_summary['missing_fraction'] = (
    atc_summary['months_zero_total'] / atc_summary['n_months']
)

In [88]:
atc_summary['ok_history']   = atc_summary['n_months'] >= 24
atc_summary['ok_volume']    = atc_summary['median_known_status_rx_atc3'] >= 50
atc_summary['ok_coverage']  = atc_summary['median_status_coverage_atc3'] >= 0.70
atc_summary['ok_missing']   = atc_summary['missing_fraction'] <= 0.20

atc_summary['keep'] = (
    atc_summary[['ok_history', 'ok_volume', 'ok_coverage', 'ok_missing']].all(axis=1)
)

atc_summary[['ok_history', 'ok_volume', 'ok_coverage', 'ok_missing', 'keep']].mean()

ok_history     0.372781
ok_volume      0.000000
ok_coverage    0.420118
ok_missing     1.000000
keep           0.000000
dtype: float64

In [89]:
atc_summary['median_known_status_rx_atc3'].describe()


count    169.000000
mean       3.289941
std        6.865976
min        0.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       43.000000
Name: median_known_status_rx_atc3, dtype: float64

In [90]:
# Use your existing atc_summary
# atc_summary has: n_months, median_known_status_rx_atc3, median_status_coverage_atc3, missing_fraction

atc_summary['ok_history']   = atc_summary['n_months'] >= 24
atc_summary['ok_volume']    = atc_summary['median_known_status_rx_atc3'] >= 5
atc_summary['ok_coverage']  = atc_summary['median_status_coverage_atc3'] >= 0.70
atc_summary['ok_missing']   = atc_summary['missing_fraction'] <= 0.20

atc_summary['keep'] = (
    atc_summary[['ok_history', 'ok_volume', 'ok_coverage', 'ok_missing']]
    .all(axis=1)
)

# See proportions that pass each rule (just for sanity)
print(atc_summary[['ok_history', 'ok_volume', 'ok_coverage', 'ok_missing', 'keep']].mean())


ok_history     0.372781
ok_volume      0.171598
ok_coverage    0.420118
ok_missing     1.000000
keep           0.053254
dtype: float64


In [91]:
atc3_pass = atc_summary.index[atc_summary['keep']].tolist()
print("ATC3 codes selected:", len(atc3_pass), "→", atc3_pass[:10])


ATC3 codes selected: 9 → ['A12A', 'ATC_UNKNOWN', 'C01E', 'J01M', 'M02A', 'N02A', 'N02B', 'N05B', 'R03D']


In [92]:
monthly_atc_filtered = monthly_atc[
    monthly_atc_full['atc_level3_for_grouping'].isin(atc3_pass)
].copy()

In [93]:
monthly_atc_filtered.head(10)

,month_start,atc_level3_for_grouping,total_rx_atc3,known_status_rx_atc3,filled_rx_atc3,status_coverage_atc3,refill_adherence_atc3,unique_patients_atc3,pct_female_atc3,median_age_atc3,low_coverage_flag_atc3
3,2023-02-01,N02B,1,1,0,1.000000,0.00,1,NaN,NaN,0
16,2023-03-01,A16A,2,1,1,0.500000,1.00,2,1.000000,64.0,1
20,2023-03-01,C01D,2,2,2,1.000000,1.00,2,0.000000,78.5,0
44,2023-03-01,J01C,5,5,5,1.000000,1.00,5,0.600000,59.0,0
45,2023-03-01,J01F,4,4,3,1.000000,0.75,3,1.000000,44.5,0
48,2023-03-01,M02A,4,4,4,1.000000,1.00,4,1.000000,57.5,0
52,2023-03-01,N02B,6,5,4,0.833333,0.80,6,0.166667,55.0,0
59,2023-03-01,N07C,2,2,2,1.000000,1.00,2,1.000000,68.0,0
67,2023-03-01,R06A,6,6,6,1.000000,1.00,5,0.666667,56.5,0
172,2023-05-01,A12C,2,1,1,0.500000,1.00,2,0.500000,49.0,1
